## Assignment 3: Dependency Parsing

### Estimated Time: ~10 hours

This assignment will build a neural dependency parser using PyTorch.  In part 1, we will review two general neural network techniques (Adam optimization and Dropout).  In part 2, we will implement and train a dependency parser using techniques from part 1.

## Part 1.  Adam Optimization and Dropout

### a) Adam

Recall the SGD update rule:

$$\theta = \theta - \alpha\triangledown_\theta J_{\text{minibatch}}(\theta)$$

where $\theta$ is a vector containing all of the model parameters, $J$ is the loss function, $\triangledown_\theta J_{\text{minibatch}}(\theta)$ is the gradient of the loss function, and $\alpha$ is the learning rate.  Adam is another possible update rule with two additional steps.

- (2 pts) First, Adam uses a trick called momentum by keep track of $\mathbf{m}$, a rolling average of the gradients:

$$\mathbf{m} = \beta_1 \mathbf{m} + (1-\beta_1)\triangledown_\theta J_{\text{minibatch}}(\theta)$$
$$\theta = \theta - \alpha \mathbf{m}$$

  where $\beta_1$ is a hyperparameter between 0 and 1 (often set to 0.9).  Briefly explain in 2-4 sentences (just give an intuition) how using $\mathbf{m}$ stops the updates from varying as much and why this low variance may be helpful to learning, overall.
  
#### <font color="red">Write your answer here.</font> 

#### <font color="blue">Solution</font> 

The momentum term increases for dimensions whose gradients point in the same directions and reduces updates for dimensions whose gradients change directions.  Gain faster convergence and reduced oscillation.

- (2 pts) Adam extends the idea of momentum with the trick of adaptive learning rates by keep track of $\mathbf{v}$, a rolling average of the magnitudes of the gradients:

$$\mathbf{m} = \beta_1 \mathbf{m} + (1 - \beta_1)\triangledown_\theta J_{\text{minibatch}}(\theta)$$
$$\mathbf{v} = \beta_2 \mathbf{v} + (1 - \beta_2)(\triangledown_\theta J_{\text{minibatch}}(\theta) \circ \triangledown_\theta J_{\text{minibatch}}(\theta))$$
$$\theta = \theta - \alpha \mathbf{m} \mathbin{/} \sqrt{\mathbf{v}}$$

where $\circ$ and $\mathbin{/}$ denote elementwise multiplication and division (not dot product!).  $\beta_2$ is a hyperparameter between 0 and 1 (often set to 0.99).  Since Adam divides the update by $\sqrt{\mathbf{v}}$, what kinds of weights will receive larger update and smaller update?  Give some simple example of how.  Why might this help with learning?

#### <font color="red">Write your answer here.</font> 

#### <font color="blue">Solution</font> 

Weights that receive high gradients will have their effective learning rate reduced and vice versa for small gradients which will receive increased learning rate. A simple example is let's say $\mathbf{v} = [9, 2, 1]$, its square root is $\sqrt{\mathbf{v}} = [3, \sqrt{2}, 1]$.  Thus by dividing $\mathbf{m}$ by this $\sqrt{\mathbf{v}}$, it is essentially scaling the bigger one to be a bit smaller, and the smaller one will become relatively larger.  This might be helpful because it can get recent stagnant parameters moving more efficiently along their axes and in turn expedite convergence.

### b) Dropout

Dropout is a regularization technique.  During training, dropout randomly sets units in the hidden layer $\mathbf{h}$ to zero with probabilty $p_{\text{drop}}$ (dropping different units each minibatch), and then multiplies $\mathbf{h}$ by a constant $\gamma$.  We can write this as:

$$\mathbf{h}_{\text{drop}} = \gamma \mathbf{d} \circ \mathbf{h}$$

where $\mathbf{d} \in \{0, 1\}^{D_h}$ ($D_h$ is the size of $\mathbf{h}$) is a mask vector where each entry is 0 with probability $p_{\text{drop}}$ and 1 with probability ($1 - p_{\text{drop}}$). For the gamma constant, $\gamma$ is chosen such that the expected value of $\mathbf{h}_{\text{drop}}$ is $\mathbf{h}$ 

$$\mathbb{E}_{\text{p_drop}}[\mathbf{h}_{\text{drop}}]_i = h_i$$

for all $i \in \{1, \cdots, D_h\}$

- (2 pts) What must $\gamma$ equal in term of $p_\text{drop}$?  Briefly justify your answers or show your math derivation using the equations given above.

#### <font color="red">Write your answer here.</font> 

#### <font color="blue">Solution</font> 

During training we drop units at a rate of $p_{\text{drop}}$, resulting in roughly $p_{\text{keep}} = 1 - p_{\text{drop}}$ fraction of units left over. At test time we’d like to have the effect of keeping a similar fraction, $p_{\text{keep}}$, of units on. By scaling down the layer units by $γ = \frac{1}{1-p_{\text{drop}}} = \frac{1}{p_{\text{keep}}}$, we effectively level out their magnitudes so that both phases of learning share very similar expected outputs.

- (2pts) Why shoup dropout be applied only during training? Why should dropout NOT be applied during evaluation?

#### <font color="red">Write your answer here.</font> 

#### <font color="blue">Solution</font> 

The goal of dropout is to reduce overfitting. We’re interested in updating unit weights so as to form a network that performs well across different datasets. Now, during evaluation we’re concerned with how well the model handles unseen data. When we dropout units, we’re “thinning” out the network which in many cases will add noise to predictions and dampen accuracy. Thus, if we were to apply dropout during evaluation time, we would not be able to fairly assess the generalization power of the network.

## Part 2.  Neural Transition-Based Dependency Parsing

We will be implementing a neural dependency parser with the goal of maximizing the performance on the UAS (Unlabeled Attachment Score) metric.

A dependency parser analyzes the grammatical structure of a sentence, establishing relationships between head words, and words which modify those heads.  There are multiple types of dependency parsers, including transition-based parsers, graph-based parsers, and feature-based parsers.  Your implementation will be a transition-based parser, which incrementally builds up a parse one step at a time.  At every step, it maintains a partial parse, which is represented as follows:

- A **stack** of words that are currently being processed
- A **buffer** of words yet to be processed.
- A **list of dependencies** predicted by the parser

Initially, the stack only contains ROOT, the dependencies list is empty, and the buffer contains all words of the sentence in order. At each step, the parser applies a tranistion to the partial parse until its buffer is empty and the stack size is 1.  The following transitions can be applied:

- $\texttt{SHIFT}$: removes the first word from the buffer and pushes it onto the stack.
- $\texttt{LEFTARC}$: marks the second (second msot recently aded) item on the stack as a dependent of the first item and removes the second item from the stack, adding a *first_word* $\rightarrow$ *second_word* dependency to the dependeny list.
- $\texttt{RIGHTARC}$: marks the first (second msot recently aded) item on the stack as a dependent of the second item and removes the first item from the stack, adding a *second_word* $\rightarrow$ *first_word* dependency to the dependeny list.

On each step, your parser will decide among the three transitions using a neural network classifier.

- (4 pts) Go through the sequence of transitions needed for parsing the sentence *I parsed this sentence correctly*.  The dependency tree for the sentence is shown below.  At each step, give the configuration of the stack and buffer, as well as what transition was applied this step and what new dependency was added (if any).  The first three steps are provided below as an example.

<img src = "img/parsetree.png" width=400>

| Stack | Buffer | New dependency | Transition |
| :--   |  :--   | :--            | :--        |
| [ROOT] | [I, parsed, this, sentence, correctly] | | Init |
| [ROOT, I] | [parsed, this, sentence, correctly] | | SHIFT |
| [ROOT, I, parsed] | [this, sentence, correctly] | | SHIFT|
| [ROOT, parsed] | [this, sentence, correctly] | parsed $\rightarrow$ I | LEFTARC |

#### <font color="red">Write your answer here.</font> 

#### <font color="blue">Solution</font> 

| Stack | Buffer | New dependency | Transition |
| :--   |  :--   | :--            | :--        |
| [ROOT] | [I, parsed, this, sentence, correctly] | | Init |
| [ROOT, I] | [parsed, this, sentence, correctly] | | SHIFT |
| [ROOT, I, parsed] | [this, sentence, correctly] | | SHIFT |
| [ROOT, parsed] | [this, sentence, correctly] | parsed $\rightarrow$ I | LEFTARC |
| [ROOT, parsed, this] | [sentence, correctly] |  | SHIFT |
| [ROOT, parsed, this, sentence] | [correctly] |  | SHIFT |
| [ROOT, parsed, sentence] | [correctly] | sentence $\rightarrow$ this | LEFTARC |
| [ROOT, parsed] | [correctly] | parsed $\rightarrow$ sentence | RIGHTARC |
| [ROOT, parsed, correctly] | [] |  | SHIFT |
| [ROOT, parsed] | [] | parsed $\rightarrow$ correctly | RIGHTARC |
| [ROOT] | [] | root $\rightarrow$ parsed | RIGHTARC |

- (2 pts) A sentence containing $n$ words will be parsed in how many steps (in terms of $n$)?  Briefly explain in 1-2 sentences why.

#### <font color="red">Write your answer here.</font> 

#### <font color="blue">Solution</font> 

In the worst case, parsing will take linear time, i.e. $O(n)$. At any step of parsing, we have two possible state transitions, either shifting a word from the buffer to the stack or clearing a dependent from the stack. Every word must spend a single step being shifted from the buffer, thus $n$ words cost $n$ shift steps. From the stack a word must be “arc”-ed over as a dependent exactly once, thus $n$ words cost $n$ “arc”-ing steps. Therefore, we have $2*n$ steps giving a cost of $O(n)$.

## Part 3: Implementation

- (6pts) Implement the <code>\_\_init\_\_</code> and <code>parse_step</code> functions in the <code>PartialParse</code> class in <code>parser_transitions.py</code>.  This implements the transition mechanics your parser will use.  You can run basic (non-exhaustive) tests by running 

- (8pts) Our network will predict which transition should be applied next to a partial parse.  We could use it to parse a single sentence by applying predicted transitions until the parse is complete.  However, neural networks run much more efficiently when making predictions about batches of data (i.e., predicting the next transition for any different partial parses simultaneously).  We can parse sentences in minibatches with the following algorithm

**Input**: <code>sentences</code>, a list of sentences to be parsed and <code>model</code>, our model that makes parse decisions

1. Initialize <code>partial_parses</code> as a list of <code>PartialParses</code>, one for each sentence in <code>sentences</code>
2. Initailize <code>unfinished_parses</code> as a shallow copy of <code>partial_parses</code>
3. **while** <code>unfinished_parses</code> is not empty **do**
    - Take the first <code>batch_size</code> parses in <code>unfinished_parses</code> as a minibatch
    - Use the <code>model</code> to predict the next transition for each partial parse in the minitbatch
    - Peform a parse step on each partial parse in the minibatch with its predicted transition
    - Remove the completed (empty buffer and stack of size 1) parses from <code>unfinished_parses</code>
    
**Return**: The dependencies for each parse in <code>partial_parses</code>

## Part 4: Evaluation